Assumption: The earnings date is taken as the ex-date

1. Get list of company tickers from finviz
2. Get earnings data report for each company in `companies`
3. Make a DF row for each company row

Columns to populate:
For each company: For each earnings
1. compute gap%
2. find C1
3. find O
4. find market cap
5. find distance from 52W high
6. find recency of 52W high
7. trading volume near announcement (at exdate)
8. trading volume before announcement (at c1)

FORMULAE:
1. Market cap = Total shares outstanding * price at that date
2. 52W_HIGH = 
3. 52W_RECENCY = 
4. 


CHANGES:
1. Change 52week defintion to 252 days instead of 364 days.
2. for the first week, do min (52 weeks, available weeks)


In [ ]:
import pandas as pd
stocks = pd.read_csv('../data/remaining.csv', header=None)

In [ ]:
ignore_filters = [ 'Closed-End Fund - Foreign', 'Closed-End Fund - Equity', 'Asset Management', 'Closed-End Fund - Debt', 'Shell Companies', 'Exchange Traded Fund']
filtered_stocks = stocks.loc[~stocks['Industry'].isin(ignore_filters)]
filtered_stocks.to_csv('stocks/FINAL_US_STOCKS.csv', mode='w', sep=',')
filtered_stocks.shape

In [ ]:
filtered_us_stocks = pd.read_csv('stocks/FINAL_US_STOCKS.csv')

In [ ]:
filtered_us_stocks.shape

In [ ]:
# GET US STOCK LIST FROM FINVIZ

from finvizfinance.screener.custom import Custom

def get_us_stocks():
    """
    Unused
    """
    fcustom = Custom()
    cols = [0, 1]
    ignore_filters = [ 'Closed-End Fund - Foreign', 'Closed-End Fund - Equity', 'Closed-End Fund - Debt', 'Shell Companies', 'Exchange Traded Fund']
    my_industries = list(set(get_fv_filter_options('Industry')) - set(ignore_filters))
#     print(my_industries)
    master_df = pd.DataFrame()
    for industry in my_industries:
        fcustom = Custom()
        cols = [0,1]
        my_filters = { 'Country': 'USA', 'Industry': industry}
        fcustom.set_filter(filters_dict=my_filters)
        df1 = fcustom.screener_view(columns=cols, sleep_sec=1, order='Ticker')
        pd.concat([master_df, df1], ignore_index=True)
        print('[+] Done {}'.format(industry))
    return master_df
#     print("stringified", ','.join(my_industries))
#     filters = { 'Country': 'USA', 'Industry': ','.join(my_industries)}
#     fcustom.set_filter(filters_dict=filters)
#     df1 = fcustom.screener_view(columns=cols, sleep_sec=1, order='Ticker')
#     return df1

def get_filters():
    fcustom = Custom()
    return fcustom.get_filters()

def write_to_file(df, filename):
    df.to_csv(rf"{filename}", header=None, sep=' ', mode='w')

def get_fv_filter_options(filtername):
    fcustom = Custom()
    return fcustom.get_filter_options(filtername)

write_to_file(get_us_stocks(), "stocks/UPDATED_US_STOCKS.txt")
print("[+] Completed Task")

In [ ]:
# PACAKGE VERSION REQUIREMENTS - YAHOOQUERY
import yahooquery as yq
import requests
import urllib3

print(yq.__version__) # >=2.3.0
print(requests.__version__) # >= 2.28.2
print(urllib3.__version__) # >= 1.26.14

In [ ]:
meta = yq.Ticker('META', asynchronous=True)

In [ ]:
df = meta.history(period="max", interval="1d")

In [ ]:
df.to_csv(fr"tmp/META_ADJUSTED_OHLC.csv", mode='w', sep=',', header=df.columns)

In [ ]:
# GET OHLC DATA READING FROM US_STOCKS in stocks/
# Note: When this was first run, we got all data (even ETFs, index funds, etc.)

import time
import os
import yahooquery as yq
import pandas as pd

def read_dir(dirname):
    lst = os.listdir(dirname)
    lst = [e.split('_')[0] for e in lst]
    return lst

no_olhc = pd.read_csv('tmp/REST.csv')
del no_olhc['Unnamed: 0']
symbols = list(no_olhc['0'])

def get_bulk_ohlc_data():
  with open("logs/ohlc_errors.txt", "w") as f:
    for symbol in symbols:
        try:
          company = yq.Ticker(symbol, asynchorous=True)
          df = company.history(period="max", interval="1d")
          df.to_csv(fr"ohlc/{symbol}_ADJUSTED_OHLC.csv", mode='w', sep=',', header=df.columns)
          print("[+] Done " + symbol)
          time.sleep(0.5) # 2000 requests/hour
        except Exception as e:
          print("An exception occured: {}".format(e))
          f.write(str(e) + "\n")
          pass

# get_bulk_ohlc_data()
        

In [ ]:
import time
import requests
import pandas as pd
import math

# MAKE DIRECTORIES earnings/, logs/, income_stmts/ before running get_data()
# This reads from FINAL_US_STOCKS.csv in stocks/
# Make sure it exists at the desired path

BASE_URL = 'https://alphavantage.co'
AV_API_KEY = 'IHLNUQ0G66C0MJID'
# ignore
AV_API_KEY_2 = 'IINQBSLFU0AKNY53'
AV_API_KEY_3 = 'L9X61VSB6U2BMPHW'
AV_API_KEY_4 = 'FSMTFWDL6XFDQSLN'
AV_API_KEY_5 = '7FMXJEH0JBVC8D7M'

# keys = [
#    AV_API_KEY,
#    AV_API_KEY_2,
#    AV_API_KEY_3,
#    AV_API_KEY_4,
#    AV_API_KEY_5
# ]

def get_bulk_earnings_data():
   """
   Reads from earnings/ to get diff
   Make sure FINAL_US_STOCKS.csv exists inside stocks/

   """
   remaining =  list(set(read_symbols('stocks/FINAL_US_STOCKS.csv')) - set(parse_symbols('earnings/')))
   print("Remaining", len(remaining))
   with open('logs/earnings_errors.txt', 'w') as f:
    for symbol in remaining:
        try:
          df = av_get_earnings(AV_API_KEY, symbol)
          if df is None:
                raise KeyError()
          df.to_csv(fr'earnings/{symbol}_EARNINGS_DATA.csv', mode='w', sep=',', header=df.columns)
          print("[+] DONE {}".format(symbol))
          time.sleep(math.ceil(12))
        except KeyError as ke:
          print('KeyError: quarterly Earnings missing for ' + str(symbol))
          f.write('KeyError: quarterly Earnings missing for ' + str(symbol))
          pass
        except Exception as e:
          print("An exception occured: {}".format(e))
          f.write(str(symbol) + '\t' + str(e) + '\n')
          pass


def av_get_adjusted_ohlc(api_key, symbol):
    """ return adjusted OHLC for symbol for the year as a dataframe"""
    url = f"{BASE_URL}/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&datatype=json&outputsize=full&apikey={api_key}"

    response = requests.get(url)

    return pd.DataFrame(response.json()['Time Series (Daily)'])

def get_bulk_us_ohlc_data(filename, start_date='', end_date=''):
    symbols = read_symbols(filename=filename)

    for symbol in symbols:
      df = av_get_adjusted_ohlc(api_key=AV_API_KEY, symbol=symbol)
      df = df.T      
      df['date'] = df.index
      if start_date and end_date:
        filtered_df = df.loc[(df['date'] > start_date) & (df['date'] < end_date)]
      else:
        filtered_df = df
      # write to a CSV file
      filtered_df.to_csv(rf"ohlc/{symbol}_ADJUSTED_OHLC.txt", header=filtered_df.columns, sep=',', mode='w')
      time.sleep(12) # limit is 5 requests / minute
      print(f"[+] {symbol} DONE")

def get_us_ohlc_data(symbol, start_date='', end_date=''):
    df = av_get_adjusted_ohlc(api_key=AV_API_KEY, symbol=symbol)
    df = df.T      
    df['date'] = df.index
    if start_date and end_date:
      filtered_df = df.loc[(df['date'] > start_date) & (df['date'] < end_date)]
    else:
      filtered_df = df
    # write to a CSV file
    filtered_df.to_csv(rf"tmp/{symbol}_ADJUSTED_OHLC.txt", header=filtered_df.columns, sep=',', mode='w')
    print(f"[+] {symbol} DONE")

def av_get_earnings(api_key, ticker):
    ticker = ticker.strip().upper()

    url = f"{BASE_URL}/query?function=EARNINGS&symbol={ticker}&apikey={api_key}"
    response = requests.get(url)
    raw_data = response.json()
    if 'quarterlyEarnings' in raw_data:
        return pd.DataFrame(raw_data['quarterlyEarnings'])
    else:
        return None

def av_get_income_statement(api_key, symbol):
    url = rf"{BASE_URL}/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={api_key}"
    response = requests.get(url)
    frame = pd.DataFrame(response.json()['quarterlyReports'])
    frame.to_csv(rf'income_stmts/{symbol}_INCOME_STATEMENT.csv', mode='w', sep=',', header=frame.columns)


remaining =  list( set(read_symbols('stocks/FINAL_US_STOCKS.csv')) - set(parse_symbols('earnings/')))
print("Remaining", len(remaining))
# get_bulk_earnings_data()
# df = av_get_earnings(AV_API_KEY, "META")
# print(df)
# earnings_data = av_get_earnings(AV_API_KEY, symbol)
# earnings_data.to_csv(f"earnings/{symbol}_EARNINGS_DATA.txt", sep=',', mode='w', header=earnings_data.columns)
# get_bulk_us_ohlc_data("tmp/US_STOCKS.txt")
# get_us_ohlc_data(symbol=symbol)
# get adjusted OHLC data for all US_stocks for the year 2022


In [ ]:
import time
import requests
import pandas as pd
import math

BASE_URL = r"https://www.alphavantage.co"
AV_API_KEY = 'IHLNUQ0G66C0MJID'  # basil api key

In [ ]:
def av_api(func, api_key, symbol):
    assert symbol is not None
    symbol = symbol.strip().upper()

    url = f"{BASE_URL}/query?function={func}&symbol={symbol}&apikey={api_key}"
    
    response = requests.get(url)
    if func == 'EARNINGS':
        return pd.DataFrame(response.json()['quarterlyEarnings'])
    elif func == 'BALANCE_SHEET':
        return pd.DataFrame(response.json()['quarterlyReports'])
    elif func == 'INCOME_STATEMENT':
        return pd.DataFrame(response.json()['quarterlyReports'])

symbol = 'META'




In [ ]:


def read_dir(dirname):
    lst = os.listdir(dirname)
    lst = [e.split('_')[0] for e in lst]
    return lst

symbols = read_dir('ohlc')

for symbol in symbols:
    try:
        df3 = av_api('BALANCE_SHEET', AV_API_KEY, symbol)
        df3.to_csv('sharesoutstanding/{}_SHARESOUTSTANDING.csv'.format(symbol), mode='w', sep=',', header=df3.columns)
        print("[+] Done {} of {}".format(symbols.index(symbol), len(symbols)))
        time.sleep(0.1)
    except Exception as e:
        print("Exception", e)



In [ ]:
import yahooquery as yq
t = yq.Ticker('aapl')
t.summary_detail

In [ ]:
import os
import time
import subprocess

def parse_symbols(dirname):
   files = os.listdir(dirname)
   return [e[:e.index('_')] for e in files]

def read_symbols(filename, remove_header=0):
    with open(filename, "r") as file:
      data = file.read().split('\n')
      data = [d.split(',')[1] for d in data if d != '']
      if remove_header == 1:
        data.pop(0) # ignore header
    return data

def get_done_symbols():
    data = parse_symbols('data/')
    data = pd.Series(data)
    data.to_csv('done/symbols.csv', mode='w', sep=',', header=None)
    return True

def get_av_data(symbol):
    #For reported Earnings & Analyst Estimated EPS, totalRevenue, commonStockSharesOutstanding
    df1 = av_api('EARNINGS',AV_API_KEY, symbol)
    df2 = av_api('INCOME_STATEMENT',AV_API_KEY, symbol)[['fiscalDateEnding','totalRevenue']]
    df1 = df1.merge(df2, on='fiscalDateEnding', how='left')
    df3 = av_api('BALANCE_SHEET',AV_API_KEY, symbol)[['fiscalDateEnding','commonStockSharesOutstanding']]
    df1 = df1.merge(df3, on='fiscalDateEnding', how='left')

    return df1

def get_bulk_data():
    symbols = read_symbols('stocks/FINAL_US_STOCKS.csv')
    remaining =  list(set(symbols) - set(parse_symbols('data/')))
    print("remaining", len(remaining))
    with open('logs/earnings_errors.txt', 'w') as f:
        for symbol in remaining:
            try:
                df = get_av_data(symbol)
                df.to_csv(fr'data/{symbol}_DATA.csv', mode='w', sep=',', header=df.columns)
                print("[+] DONE {}: {} of {}".format(symbol, remaining.index(symbol), len(remaining)))
                time.sleep(1)
            except KeyError as ke:
                print('KeyError: quarterly Earnings missing for ' + str(symbol))
                f.write('KeyError: quarterly Earnings missing for ' + str(symbol))
                pass
            except Exception as e:
                print("An exception occured: {}".format(e))
                f.write(str(symbol) + '\t' + str(e) + '\n')
                pass


def network_check():
    while True:
        def check_ping():
            hostname = "8.8.8.8"
            response = subprocess.call(['runas', '/user:Administrator', "ping -c 1 " + hostname])
            print(response)
            if response == 1:
                pingstatus = 0
            else: pingstatus = 1

            return pingstatus
        
        def get_bulk_data():
            symbols = read_symbols('stocks/FINAL_US_STOCKS.csv', remove_header=1)
            done = read_symbols('done/symbols.csv')
            remaining =  list(set(symbols) - set(done))
            print("remaining", len(remaining))
            with open('logs/earnings_errors.txt', 'w') as f:
                for symbol in remaining:
                    try:
                        df = get_av_data(symbol)
                        df.to_csv(fr'data/{symbol}_DATA.csv', mode='w', sep=',', header=df.columns)
                        print("[+] DONE {}: {} of {}".format(symbol, remaining.index(symbol), len(remaining)))
                        time.sleep(0.01)
                    except KeyError as ke:
                        print('KeyError: quarterly Earnings missing for ' + str(symbol))
                        f.write('KeyError: quarterly Earnings missing for ' + str(symbol))
                        pass
                    except Exception as e:
                        print("An exception occured: {}".format(e))
                        f.write(str(symbol) + '\t' + str(e) + '\n')
                        pass
        
        net_stat = check_ping()

        if net_stat == 0:
            get_bulk_data()
        else:
            print("Resetting network adapters")
            dwnnw = "ifconfig wlan0 down"
            upnw = "ifconfig wlan0 up"
            os.system(dwnnw)
            os.system(upnw)
            time.sleep(1)
            continue

# get_done_symbols()

all = pd.read_csv('stocks/FINAL_US_STOCKS.csv')
done = pd.read_csv('done/symbols.csv')

remaining = list(set(all['Ticker']) - set(done['Ticker']))
pd.Series(remaining).to_csv('done/remaining.csv')

In [ ]:
symbols = read_symbols('stocks/FINAL_US_STOCKS.csv', remove_header=1)
done = read_symbols('done/symbols.csv')
remaining =  list(set(symbols) - set(done))
print(len(remaining))
print(remaining)
remaining_df = pd.Series(remaining)
remaining_df.to_csv('done/remaining.csv', mode='w', sep=',')

In [ ]:
df

1. Get all OHLC data for 7k companies
2. Get earnings data for each company in ohlc/
3. Get company overview for each company in ohlc/
4. 

In [ ]:
import datetime

# def get_bulk_us_earnings_data(filename):
#     """
#     filename should be 'stocks/FINAL_US_STOCKS.csv
#     """
#     symbols = read_symbols(filename=filename)

#     for symbol in symbols:
#         earnings_data = av_get_earnings(api_key=AV_API_KEY, ticker=symbol)
#         earnings_data.to_csv(
#             f"earnings/{symbol}_EARNINGS_DATA.txt", sep=',', mode='w', header=earnings_data.columns)
#         time.sleep(12)

def av_get_company_overview(api_key, symbol):
    """
    deprecated
    """
    symbol = symbol.strip().upper()

    url = f"{BASE_URL}/query?function=OVERVIEW&symbol={symbol}&apikey={api_key}"
    response = requests.get(url)

    return response.json()


def get_bulk_us_overview(filename):
    symbols = read_symbols(filename=filename)

    for symbol in symbols:
        overview = av_get_company_overview(api_key=AV_API_KEY, symbol=symbol)
        with open(f"overview/{symbol}_OVERVIEW.json", "w") as f:
            f.write(overview)
        time.sleep(12)


def build_gap_table(filename):
    """
    deprecated
    """
    symbols = read_symbols(filename=filename)

    for symbol in symbols:
        ohlc_df = pd.read_csv(f"ohlc/{symbol}_ADJUSTED_OHLC.txt")
        earnings_dates = pd.read_csv(f"earnings/{symbol}_EARNINGS_DATA.txt")
        ohlc_df = ohlc_df.set_index('date')
        openings_at_ex_date_df = ohlc_df.loc[ohlc_df.index.isin(
            earnings_dates)]
        overview_json = pd.read_json(f"overview/{symbol}_OVERVIEW.txt")

        myrows = []
        # for each earnings date
        for index, row in openings_at_ex_date_df.iterrows():
            # find the closing date data before exdate
            if ohlc_df.index[ohlc_df['Unnamed: 0'] == row['Unnamed: 0']].tolist()[0]:
                c1 = ohlc_df.iloc[ohlc_df.index.get_loc(
                    ohlc_df.index[ohlc_df['Unnamed: 0'] == row['Unnamed: 0']].tolist()[0])+1]
            mydict = {}
            mydict['exdate'] = index
            mydict['Opening at ex date'] = row['1. open']
            mydict['C1 date'] = c1['Unnamed: 0']
            mydict['C1'] = c1['4. close']
            mydict['Gap%'] = (row['1. open'] / c1['4. close'] - 1) * 100
            mydict['Market Cap'] = overview_json['MarketCapitalization']
            mydict['52W_dist'] = c1['4. close'] - float(overview_json['52WeekHigh'])

            fiftytwowh_date_series = ohlc_df.loc[ohlc_df['2. high'] == float(
                overview_json['52WeekHigh'])]['Unnamed: 0']

            # TODO: Refactor
            if fiftytwowh_date_series is not None:
                c1_date = datetime.datetime.strptime(
                    c1['Unnamed: 0'], "%Y-%m-%d").date()
                fiftytwoWH_date = datetime.datetime.strptime(
                    fiftytwowh_date_series[0], "%Y-%m-%d").date()

                # doesn't make sense to calculate recency if NOT from the same year
                if c1_date.year == fiftytwoWH_date.year:
                    recency = c1_date - fiftytwoWH_date
                else:  
                    recency = ''
            else:
                recency = ''
            
            mydict['52W_recency'] = recency
            mydict['Trading vol at exdate'] = row['6. volume']

            myrows.append(mydict)
        gap = pd.DataFrame(myrows)
        gap.to_csv(f"{symbol}_GAP_TABLE.txt", header=gap.columns, sep=',', mode='w')
        print(f"[+] Constructed gap table for ${symbol}")


In [ ]:
symbol = "META"
earnings_dates = av_get_earnings(AV_API_KEY, symbol)['reportedDate']
type(earnings_dates)

In [ ]:
get_us_ohlc_data("META")

In [ ]:
ohlc_df = pd.read_csv(f"ohlc/{symbol}_ADJUSTED_OHLC.txt")

In [ ]:
ohlc_df = ohlc_df.set_index('date')
openings_at_ex_date_df = ohlc_df.loc[ohlc_df.index.isin(earnings_dates)]

In [ ]:
def av_get_company_overview(api_key, symbol):
    symbol = symbol.strip().upper()
    
    url = f"{BASE_URL}/query?function=OVERVIEW&symbol={symbol}&apikey={api_key}"
    response = requests.get(url)

    return response.json()

In [ ]:
overview_json = av_get_company_overview(AV_API_KEY, "META")

In [ ]:
symbol = "META"
def get_52_week_data(date):
    """
    Return the 52 week (252 days) 
    Note: This function ignores the first 51 weeks of the historical OHLC as 52Wh is not defined
    """
    myrows = []
    for i in range(252):
      try:
        c1 = ohlc_df.iloc[ohlc_df.index.get_loc(ohlc_df.index[ohlc_df['Unnamed: 0'] == date].tolist()[0]) + i]
        myrows.append(c1)
      except IndexError:
         pass
    frame = pd.DataFrame(myrows)
    high = max(frame['2. high'])
    high_date = frame['2. high'].idxmax()
    low_date = frame['3. low'].idxmin()
    low = min(frame['3. low'])
    wh52 = frame.index.get_indexer_for([high_date])
    wl52 = frame.index.get_indexer_for([low_date])
    return (high, int(wh52), low, int(wl52))

def get_65_day_low(date):
    myrows = []
    ohlc_df = pd.read_csv(fr"ohlc/{symbol}_ADJUSTED_OHLC.txt")
    for i in range(65):
      try:
        c1 = ohlc_df.iloc[ohlc_df.index.get_loc(ohlc_df.index[ohlc_df['Unnamed: 0'] == date].tolist()[0]) + i]
        myrows.append(c1)
      except IndexError:
        pass
    return min(pd.DataFrame(myrows)['4. close'])
get_65_day_low('2023-02-01')

1. Ex date
2. O
3. H
4. L
5. C
6. C1 date
7. C1
8. Gap%
9. Mkt. Cap
10. 52WH dist
11. 52WH recency
12. 52WL dist
13. 52WL recency
14. 65DL dist
15. Day of the week
16. Avg 20 days volume
17. v0
18. v1
19. fundamentals

In [ ]:
def get_avg_volume(symbol="META", start="", days=20):
    if start is None:
      return None
    ohlc_df = pd.read_csv(fr"ohlc/{symbol}_ADJUSTED_OHLC.csv")
    myrows = []
    for i in range(days):
      try:
        vols = ohlc_df.iloc[ohlc_df.index.get_loc(ohlc_df.index[ohlc_df['Unnamed: 0'] == start].tolist()[0]) + i]
        myrows.append(vols['6. volume'])
      except IndexError:
         pass      
    return pd.Series(myrows).mean()

get_avg_volume("META", "2023-02-01")

In [ ]:
earnings_data = pd.read_csv(fr"earnings/{symbol}_EARNINGS_DATA.csv")

In [ ]:
def av_get_income_statement(api_key, symbol):
    url = rf"{BASE_URL}/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={api_key}"
    response = requests.get(url)
    frame = pd.DataFrame(response.json()['quarterlyReports'])
    frame.to_csv(rf'income_stmts/{symbol}_INCOME_STATEMENT.csv', mode='w', sep=',', header=frame.columns)

def get_fundamental_data(symbol, exdate):
    earnings = pd.read_csv(fr"earnings/{symbol}_EARNINGS_DATA.txt")
    income_stmts = pd.read_csv(fr"income_stmts/{symbol}_INCOME_STATEMENT.csv")
    row = earnings[earnings['reportedDate'] == exdate]

    reported_eps = row[['reportedEPS', 'estimatedEPS', 'surprise']]
    # return type(float(reported_eps))
    return reported_eps
    # return (reported_eps, estimated_eps, surprise)
    # return (float(reported_eps), float(estimated_eps), float(surprise))
get_fundamental_data('META','2023-02-01')


In [ ]:
import datetime
import calendar
myrows = []
for index, row in openings_at_ex_date_df.iterrows():
    if ohlc_df.index[ohlc_df['Unnamed: 0'] == row['Unnamed: 0']].tolist()[0]:
        c1 = ohlc_df.iloc[ohlc_df.index.get_loc(
            ohlc_df.index[ohlc_df['Unnamed: 0'] == row['Unnamed: 0']].tolist()[0])+1]
    mydict = {}
    mydict['exdate'] = index
    mydict['weekday'] = calendar.day_name[datetime.datetime.strptime(index, "%Y-%m-%d").date().weekday()]
    mydict['O'] = row['1. open']
    mydict['H'] = row['2. high']
    mydict['L'] = row['3. low']
    mydict['C'] = row['4. close']
    mydict['C1 date'] = c1['Unnamed: 0']
    mydict['C1'] = c1['4. close']
    mydict['Gap%'] = (row['1. open'] / c1['4. close'] - 1) * 100

    # TODO: Market cap
    # 1. Market cap = Total shares outstanding * price at that date
    mydict['MktCap'] = float(overview_json['SharesOutstanding']) * float(row['1. open'])

    # TODO: Compute 52WH from OHLC data (52*7)
    # Formula: (52WH - Close) / 52WH
    WH_52, WH52_recency, WL_52, WL52_recency = get_52_week_data(c1['Unnamed: 0'])
    DL_65 = get_65_day_low(c1['Unnamed: 0'])
    # print(WH_52, WH52_recency, WL_52, WL52_recency, "C1", c1['Unnamed: 0'])

    mydict['52WH_dist'] = (WH_52 - c1['4. close']) / WH_52
    mydict['52WH_recency'] = WH52_recency
    mydict['52WL_dist'] = (c1['4. close'] - WH_52) / WL_52
    mydict['52WL_recency'] = WL52_recency

    # TODO: check validity
    mydict['65DL_dist'] = (c1['4. close'] - DL_65) / DL_65

    # TODO: 52WH_recency
    # Redo with respect to new defintion
    # fiftytwowh_date_series = ohlc_df.loc[ohlc_df['2. high'] == float(
    #     overview_json['52WeekHigh'])]['Unnamed: 0']
    # if fiftytwowh_date_series is not None:
    #     c1_date = datetime.datetime.strptime(
    #         c1['Unnamed: 0'], "%Y-%m-%d").date()
    #     fiftytwoWH_date = datetime.datetime.strptime(
    #         fiftytwowh_date_series[0], "%Y-%m-%d").date()
    #     if c1_date.year == fiftytwoWH_date.year:
    #         recency = c1_date - fiftytwoWH_date
    #     else:
    #         recency = ''
    # else:
    #     recency = ''


    mydict['v0'] = row['6. volume']
    mydict['v1'] = c1['6. volume']
    mydict['avg_v20'] = get_avg_volume(symbol="META", start=c1['Unnamed: 0'], days=20)
    # TODO: Get vol1 as well

    # FUNDAMENTALS
    fundamental_data = get_fundamental_data(symbol='META', exdate=row['Unnamed: 0'])
    # print(type(fundamental_data['estimatedEPS']))
    mydict['reportedEPS'] = fundamental_data['reportedEPS']
    mydict['estimatedEPS'] = fundamental_data['estimatedEPS']
    mydict['surprise'] = fundamental_data['surprise']

    myrows.append(mydict)

gap = pd.DataFrame(myrows)


In [ ]:
gap

In [ ]:
gap.to_csv("tmp/META_GAP_TABLE.csv", mode='w', sep=',', header=gap.columns)

In [ ]:
gap = pd.read_csv(rf"tmp/{symbol}_GAP_TABLE.csv")

In [ ]:
gap